In [10]:
import csv

class RxNormCrosswalk:
    def __init__(self, ndc_rrf_file, rxnorm_rrf_file):
        self.ndc_rrf_file = ndc_rrf_file
        self.rxnorm_rrf_file = rxnorm_rrf_file
        self.ndc_to_rxcui = {}
        self.rxcui_to_drug_class = {}

    def load_ndc_mapping(self):
        with open(self.ndc_rrf_file, 'r', encoding='utf-8') as file:
            reader = csv.reader(file, delimiter='|')
            for row in reader:
                if row[-6] == 'NDC':
                    ndc = row[-4].strip()
                    rxcui = row[0].strip()
                    if ndc and rxcui:
                        self.ndc_to_rxcui[ndc] = rxcui

    def load_drug_class_mapping(self):
        with open(self.rxnorm_rrf_file, 'r', encoding='utf-8') as file:
            reader = csv.reader(file, delimiter='|')
            for row in reader:
                rxcui = row[0].strip()
                drug_class = row[1].strip()
                if rxcui and drug_class:
                    self.rxcui_to_drug_class[rxcui] = drug_class

    def get_rxcui_for_ndc(self, ndc):
        return self.ndc_to_rxcui.get(ndc)

    def get_drug_class_for_rxcui(self, rxcui):
        return self.rxcui_to_drug_class.get(rxcui)
    

In [11]:
rxnsat_file = '/Users/prs/Downloads/RxNorm_full_04012024/rrf/RXNSAT.RRF' # RXNSAT file , ATV column with ATN == NDC
rxnconso_file = '/Users/prs/Downloads/RxNorm_full_04012024/rrf/RXNCONSO.RRF' # RXNCONSO

In [12]:
crosswalk = RxNormCrosswalk(rxnsat_file, rxnconso_file)

In [13]:
crosswalk.load_ndc_mapping()

In [15]:
ndc_to_rxcui_dict=crosswalk.ndc_to_rxcui

In [16]:
ndc_to_rxcui_df = pd.DataFrame(ndc_to_rxcui_dict.items())
ndc_to_rxcui_df.columns = ['NDC', 'RXCUI']

In [5]:
rxcui = crosswalk.get_rxcui_for_ndc('000395021301')

In [17]:
import csv
atcMapping = {'rxcui':[],'sab':[],}
with open(rxnconso_file,'r') as f:
    reader = csv.reader(f,delimiter='|')
    for row in reader:
        rows.append(row[-8])
    

In [3]:
import sqlite3

In [4]:
DB_FILE = '/Users/prs/Downloads/RxNorm_full_04012024/rrf/your_database.db'
# Database connection
conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

In [6]:
import pandas as pd 
RXNCONSO = pd.DataFrame(cursor.execute('select * from RXNCONSO').fetchall(), 
             columns= ['RXCUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'RXAUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF'])


In [28]:
RXNCONSO[RXNCONSO.RXCUI.isin(ndc_to_rxcui_df.RXCUI.tolist())].SUPPRESS.unique()

array(['N', 'O', 'E'], dtype=object)

In [7]:
query = """
select k.rxcui, k.str, m.code, m.str, n.code, n.str, o.code, o.str, f.code, f.str 
from rxnconso k, rxnrel y, rxnrel z, 
rxnconso a, rxnrel b, rxnrel c, rxnrel d, rxnrel e,
rxnconso f, rxnconso m, rxnconso n, rxnconso o 
where k.tty in ('SCD') and k.suppress = 'N'
and k.rxcui = y.rxcui2
and y.sab = 'RXNORM'			
and y.rela = 'consists_of'
and y.rxcui1 = z.rxcui2
and z.sab = 'RXNORM'
and z.rela = 'has_ingredient'
and z.rxcui1 = a.rxcui
and a.sab = 'ATC'
and a.rxaui = b.rxaui2
and b.sab = 'ATC'
and b.rela = 'member_of'  --   ATC 4 level (narrowest)
and b.rxaui1 = c.rxaui2
and c.sab = 'ATC'
and c.rela = 'isa' --  ATC 3 level 
and c.rxaui1 = d.rxaui2
and d.sab = 'ATC'
and d.rela = 'isa' --  ATC 2 level
and d.rxaui1 = e.rxaui2
and e.sab = 'ATC'
and e.rela = 'isa'  --   ATC 1 level (broadest)
and e.rxaui1 = f.rxaui
and b.rxaui1 = m.rxaui
and c.rxaui1 = n.rxaui
and d.rxaui1 = o.rxaui
"""
mapping = pd.DataFrame(cursor.execute(query).fetchall(), 
                       columns=['rxcui', 'rxn_label','ATC4_code','ATC4_label','ATC3_code','ATC3_label','ATC2_code','ATC2_label','ATC1_code','ATC1_label'])

In [17]:
query2="""
select k.rxcui, k.str, m.code, m.str
from rxnconso k, rxnrel y, rxnrel z, 
rxnconso a, rxnrel b, rxnrel c, rxnrel d, rxnrel e,
rxnconso f, rxnconso m, rxnconso n, rxnconso o 
where k.rxcui = y.rxcui2
and y.sab = 'RXNORM'			
and y.rela = 'consists_of'
and y.rxcui1 = z.rxcui2
and z.sab = 'RXNORM'
and z.rela = 'has_ingredient'
and z.rxcui1 = a.rxcui
and a.sab = 'ATC'
and a.rxaui = b.rxaui2
and b.sab = 'ATC'
and b.rela = 'member_of'  --   ATC 4 level (narrowest)
and b.rxaui1 = c.rxaui2
and c.sab = 'ATC'
and e.rxaui1 = f.rxaui
and b.rxaui1 = m.rxaui
and c.rxaui1 = n.rxaui
and d.rxaui1 = o.rxaui
"""

In [18]:
pd.DataFrame(cursor.execute(query2).fetchall())

In [10]:
mapping.columns = ['rxcui', 'rxn_label','ATC4_code','ATC4_label','ATC3_code','ATC3_label','ATC2_code','ATC2_label','ATC1_code','ATC1_label']

In [11]:
mapping

,rxcui,rxn_label,ATC4_code,ATC4_label,ATC3_code,ATC3_label,ATC2_code,ATC2_label,ATC1_code,ATC1_label
0,91348,hydrogen peroxide 300 MG/ML Topical Solution,D11AX,Other dermatologicals,D11A,OTHER DERMATOLOGICAL PREPARATIONS,D11,OTHER DERMATOLOGICAL PREPARATIONS,D,DERMATOLOGICALS
1,91348,hydrogen peroxide 300 MG/ML Topical Solution,A01AB,Antiinfectives and antiseptics for local oral ...,A01A,STOMATOLOGICAL PREPARATIONS,A01,STOMATOLOGICAL PREPARATIONS,A,ALIMENTARY TRACT AND METABOLISM
2,91348,hydrogen peroxide 300 MG/ML Topical Solution,D08AX,Other antiseptics and disinfectants,D08A,ANTISEPTICS AND DISINFECTANTS,D08,ANTISEPTICS AND DISINFECTANTS,D,DERMATOLOGICALS
3,91348,hydrogen peroxide 300 MG/ML Topical Solution,S02AA,Antiinfectives,S02A,ANTIINFECTIVES,S02,OTOLOGICALS,S,SENSORY ORGANS
4,91349,hydrogen peroxide 30 MG/ML Topical Solution,D11AX,Other dermatologicals,D11A,OTHER DERMATOLOGICAL PREPARATIONS,D11,OTHER DERMATOLOGICAL PREPARATIONS,D,DERMATOLOGICALS
...,...,...,...,...,...,...,...,...,...,...
33055,2677688,hydrocortisone 20 MG/ML Topical Cream,H02AB,Glucocorticoids,H02A,"CORTICOSTEROIDS FOR SYSTEMIC USE, PLAIN",H02,CORTICOSTEROIDS FOR SYSTEMIC USE,H,"SYSTEMIC HORMONAL PREPARATIONS, EXCL. SEX HORM..."
33056,2677688,hydrocortisone 20 MG/ML Topical Cream,S01BA,"Corticosteroids, plain",S01B,ANTIINFLAMMATORY AGENTS,S01,OPHTHALMOLOGICALS,S,SENSORY ORGANS
33057,2677688,hydrocortisone 20 MG/ML Topical Cream,S01CB,Corticosteroids/antiinfectives/mydriatics in c...,S01C,ANTIINFLAMMATORY AGENTS AND ANTIINFECTIVES IN ...,S01,OPHTHALMOLOGICALS,S,SENSORY ORGANS
33058,2677688,hydrocortisone 20 MG/ML Topical Cream,S02BA,Corticosteroids,S02B,CORTICOSTEROIDS,S02,OTOLOGICALS,S,SENSORY ORGANS
